<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/gp_new_model_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [3]:
import torch
import numpy as np

## CPU
device = torch.device("cpu")

## GPU
# device = torch.device("cuda")

In [4]:
!pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595749 sha256=c44df2e020e06c31831bd2d84a394a2ca7f439ec0293004eaf2a59460a55c568
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [5]:
!pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [6]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-v1qnjw3p
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-v1qnjw3p
     |████████████████████████████████| 129 kB 4.1 MB/s 
     |████████████████████████████████| 54.7 MB 9.5 MB/s 
     |████████████████████████████████| 4.5 MB 10.0 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x39f5e000 @  0x7f7ca8be7615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 

In [7]:
# 사용자 input

# 서버에서 본문 받아오기
# 서버에서 아마 csv나 json으로 받아올텐데, 받아올 때 특수문자 처리 필수 (특히 따옴표 '', "")
# '[제목] 본문' 형태로 전처리
# //////////////////////서버로부터 본문 받아와서 str로 저장하는 line////////////////////////
input_text= "[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다!"

if len(input_text) + len(" TL;DR ") > 800:
  input_text = input_text[0:400] + input_text[len(input_text)-400:len(input_text)]

* BERT

In [8]:
from kobert import get_pytorch_kobert_model

bertmodel, vocab  = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
import gluonnlp as nlp
from kobert import get_tokenizer

bert_tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(bert_tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/Colab Notebooks/graduation_project/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
from torch import nn

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [14]:
bert_model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [15]:
# model load
bert_model.load_state_dict(torch.load('./checkpoint/bert_model.pt', map_location=device))

<All keys matched successfully>

In [ ]:
max_len = 512

In [ ]:
# 기쁨:0 분노:1 혐오:2 두려움:3 슬픔:4 unknown:5

def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["기쁨", "분노", "혐오", "두려움", "슬픔", "unknown"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print("일기의 감정:", cate[idx])
    print("신뢰도:", "{:.2f}%".format(softmax(result,idx)))
    return cate[idx]

In [ ]:
emotion= testModel(bert_model, input_text)

일기의 감정: 슬픔
신뢰도: 19.42%


* KoGPT

In [ ]:
from transformers import GPT2LMHeadModel

gpt_model= GPT2LMHeadModel.from_pretrained('./checkpoint/gpt_model_1.pt')

In [ ]:
from transformers import PreTrainedTokenizerFast

gpt_tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
# input_text= "[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다!"
input_text= input_text + " TL;DR "
text= emotion + '| ' + input_text
# print(text)

In [ ]:
input_ids = gpt_tokenizer.encode(text, return_tensors='pt')
gen_ids = gpt_model.generate(input_ids,
                           max_length=len(input_ids[0])+ 50, # 50 -> 약 120자
                           repetition_penalty=2.0,
                           pad_token_id=gpt_tokenizer.pad_token_id,
                           eos_token_id=gpt_tokenizer.eos_token_id,
                           bos_token_id=gpt_tokenizer.bos_token_id,
                           use_cache=True).to(device)
generated = gpt_tokenizer.decode(gen_ids[0])
# print(generated)

In [ ]:
TLDR_index= generated.index('TL;DR')
output= generated[TLDR_index+6:]
print('return comment:' , output)

return comment:  그날의 일이 있었다. 나는 언제나 좋은 일을 했다. 그리고 내가 너무 힘들었다. 하지만 그렇게 생각해요. 그래도 그랬으니까 모른지도 못 쓰는 것은 물론 매일 아침이 되었다. 그러나 이제 엄마냥 한 편하기 바랄 수 없는 것 같아요 당화가 되어
